In [2]:
from pydantic import BaseModel

from ollama import Client
model = "qwen2.5:0.5b"
# model = "ollama3"
client = Client(host="http://localhost:11434")


# Define the schema for the response
class FriendInfo(BaseModel):
  name: str
  age: int
  is_available: bool


class FriendList(BaseModel):
  friends: list[FriendInfo]


# schema = {'type': 'object', 'properties': {'friends': {'type': 'array', 'items': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'age': {'type': 'integer'}, 'is_available': {'type': 'boolean'}}, 'required': ['name', 'age', 'is_available']}}}, 'required': ['friends']}
response = client.chat(
  model=model,
  messages=[{'role': 'user', 'content': 'I have two friends. The first is Ollama 22 years old busy saving the world, and the second is Alonso 23 years old and wants to hang out. Return a list of friends in JSON format'}],
  format=FriendList.model_json_schema(),  # Use Pydantic to generate the schema or format=schema
  options={'temperature': 0},  # Make responses more deterministic
)

# Use Pydantic to validate the response
friends_response = FriendList.model_validate_json(response.message.content)
print(friends_response)

friends=[FriendInfo(name='Ollama', age=22, is_available=True), FriendInfo(name='Alonso', age=23, is_available=False)]


In [3]:
FriendList.model_json_schema()

{'$defs': {'FriendInfo': {'properties': {'name': {'title': 'Name',
     'type': 'string'},
    'age': {'title': 'Age', 'type': 'integer'},
    'is_available': {'title': 'Is Available', 'type': 'boolean'}},
   'required': ['name', 'age', 'is_available'],
   'title': 'FriendInfo',
   'type': 'object'}},
 'properties': {'friends': {'items': {'$ref': '#/$defs/FriendInfo'},
   'title': 'Friends',
   'type': 'array'}},
 'required': ['friends'],
 'title': 'FriendList',
 'type': 'object'}